In [1]:
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-tf-plugin-cuda110

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://developer.download.nvidia.com/compute/redist
     |████████████████████████████████| 382.6 MB 42 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://developer.download.nvidia.com/compute/redist
     |████████████████████████████████| 418 kB 706 kB/s 
  Created wheel for nvidia-dali-tf-plugin-cuda110: filename=nvidia_dali_tf_plugin_cuda110-1.19.0-cp37-cp37m-linux_x86_64.whl size=120499 sha256=9f5a68dad5baad683d0e7b2bdf05e8ece94fb038625c33b696fff882872bd658
  Stored in directory: /root/.cache/pip/wheels/7f/8c/08/7adf9f8b758908640a1a1569489d8ab08a81a05192aa508363
Successfully built nvidia-dali-tf-plugin-cuda110


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rn
import os
from sklearn.metrics import accuracy_score, f1_score
from keras.models import load_model

import warnings
import logging

logging.getLogger('tensorflow').disabled = True
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import nvidia.dali.tfrecord as tfrec
import nvidia.dali.plugin.tf as dali_tf

In [29]:
def thresholdAccur(th, pred_test_data, true_test_data):
  if th ==0 :
    true_test_ls = []
    pred_test_ls = []
    for (true, pred) in zip(true_test, pred_test):
        true_test_ls.append(np.argmax(true))
        pred_test_ls.append(np.argmax(pred))
    argmax_acc = accuracy_score(true_test_ls, pred_test_ls)
    f1_micro = f1_score(true_test_ls, pred_test_ls, average='micro')
    print("thresholde :   - "," accuracy_score : ", argmax_acc, " f1_score : ", f1_micro)
  else:
    pred = pred_test_data.copy()
    for ls in pred:
        thres = th
        pred[pred >= thres] = 1
        pred[pred < thres] = 0
    print("thresholde : ", th, " accuracy_score : ", accuracy_score(true_test_data, pred), " f1_score : ", f1_score(true_test_data, pred, average='micro'))


In [6]:
f_PATH = "/content/drive/MyDrive/facial_emotion_recognition_using_K-drama_dataset-main/"

# 6label

In [7]:
model1 = load_model(f'/content/drive/MyDrive/facial_emotion_recognition_using_K-drama_dataset-main/model/cnn_6.h5')
model2 = load_model(f'/content/drive/MyDrive/facial_emotion_recognition_using_K-drama_dataset-main/model/resnet_6.h5')

In [12]:
set_seed = 46
np.random.seed(set_seed)
rn.seed(set_seed)
tf.random.set_seed(set_seed)
label_test = pd.read_csv(f_PATH+'label/label_6_basic_emo_test.csv')
TEST_DATA_SIZE = label_test.shape[0]

In [15]:
class TFRecordPipelineTest(Pipeline):
    def __init__(self, batch_size, num_threads,
                 device = 'cpu', device_id = 0):
        super(TFRecordPipelineTest, self).__init__(batch_size,
                                         num_threads,
                                         device_id)
        self.input = fn.readers.tfrecord(        
             features = {"image_raw": tfrec.FixedLenFeature((), tfrec.string, ""),
                         "label": tfrec.FixedLenFeature([6], tfrec.int64,  6)},
             path = [f_PATH+'tfrecords/6_basic_emo_test.tfrecords'],
             index_path = [f_PATH+'tfrecords/6_basic_emo_test.idx'],
             random_shuffle=False,
             seed = set_seed)
        self.iter = 0
    def define_graph(self):
        inputs = self.input
        images = fn.decoders.image(inputs["image_raw"], device = "mixed", output_type = types.RGB) / 255.
        labels = inputs["label"].gpu()
        return (images, labels)
    def iter_setup(self):
        pass

In [19]:
if __name__ == "__main__":
    batch_size = 1
    shapes = ((batch_size, 64, 64, 3),
              (batch_size, 6))
    pipe_test = TFRecordPipelineTest(batch_size=batch_size,
                            num_threads=4,
                            device='gpu',
                            device_id=0)
    with tf.device('/gpu:0'):
    #with strategy.scope():        
        # Create dataset
        ds_test = dali_tf.DALIDataset(
            pipeline=pipe_test,
            batch_size=batch_size,
            output_shapes=shapes,
            output_dtypes=(tf.float32, tf.int64),
            device_id=0)
        loss, acc1 = model1.evaluate(ds_test, steps=TEST_DATA_SIZE)
        print("Test CNN accuracy:", acc1)

1282/1282 [==============================] - 3s 3ms/step - loss: 0.4088 - accuracy: 0.8510
Test CNN accuracy: 0.8510140180587769


In [20]:
pred_test_cnn = model1.predict(ds_test.take(TEST_DATA_SIZE))

1282/1282 [==============================] - 3s 2ms/step


In [21]:
# https://docs.nvidia.com/deeplearning/dali/user-guide/docs/plugins/tensorflow_plugin_api.html
if __name__ == "__main__":
    batch_size = 1
    shapes = ((batch_size, 64, 64, 3),
              (batch_size, 6))
    pipe_test = TFRecordPipelineTest(batch_size=batch_size,
                            num_threads=4,
                            device='gpu',
                            device_id=0)
    with tf.device('/gpu:0'):
        ds_test = dali_tf.DALIDataset(
            pipeline=pipe_test,
            batch_size=batch_size,
            output_shapes=shapes,
            output_dtypes=(tf.float32, tf.int64),
            device_id=0)
        loss, acc2 = model2.evaluate(ds_test, steps=TEST_DATA_SIZE)
        print("Test ResNet accuracy:", acc2)

1282/1282 [==============================] - 12s 9ms/step - loss: 0.2066 - accuracy: 0.9750
Test ResNet accuracy: 0.975039005279541


In [22]:
pred_test_resnet = model2.predict(ds_test.take(TEST_DATA_SIZE))

1282/1282 [==============================] - 10s 8ms/step


In [23]:
pred_test = (pred_test_cnn * (acc1/(acc1+acc2)) )+(pred_test_resnet * (acc2/(acc1+acc2)))
true_test = np.concatenate([y for x, y in ds_test.take(TEST_DATA_SIZE)], axis=0)

In [30]:
thresholdAccur(0, pred_test, true_test)
thresholdAccur(0.1, pred_test, true_test)
thresholdAccur(0.2, pred_test, true_test)
thresholdAccur(0.3, pred_test, true_test)
thresholdAccur(0.4, pred_test, true_test)
thresholdAccur(0.5, pred_test, true_test)

thresholde :   -   accuracy_score :  0.9750390015600624  f1_score :  0.9750390015600624
thresholde :  0.1  accuracy_score :  0.719188767550702  f1_score :  0.8585858585858585
thresholde :  0.2  accuracy_score :  0.8198127925117005  f1_score :  0.9087656529516994
thresholde :  0.3  accuracy_score :  0.8985959438377535  f1_score :  0.943593574897273
thresholde :  0.4  accuracy_score :  0.9500780031201248  f1_score :  0.9654112221368177
thresholde :  0.5  accuracy_score :  0.9703588143525741  f1_score :  0.9760690466849745


# 22 label

In [31]:
model1 = keras.models.load_model(f'/content/drive/MyDrive/facial_emotion_recognition_using_K-drama_dataset-main/model/cnn_22.h5')
model2 = keras.models.load_model(f'/content/drive/MyDrive/facial_emotion_recognition_using_K-drama_dataset-main/model/resnet_22.h5')

In [32]:
class TFRecordPipelineTest(Pipeline):
    def __init__(self, batch_size, num_threads,
                 device = 'cpu', device_id = 0):
        super(TFRecordPipelineTest, self).__init__(batch_size,
                                         num_threads,
                                         device_id)
        self.input = fn.readers.tfrecord(        
             features = {"image_raw": tfrec.FixedLenFeature((), tfrec.string, ""),
                         "label": tfrec.FixedLenFeature([22], tfrec.int64,  22)},
             path = [f_PATH+'tfrecords/22_emo_test.tfrecords'],
             index_path = [f_PATH+'tfrecords/22_emo_test.idx'],
             random_shuffle=False,
             seed = set_seed)
        self.iter = 0
    def define_graph(self):
        inputs = self.input
        images = fn.decoders.image(inputs["image_raw"], device = "mixed", output_type = types.RGB) / 255.
        labels = inputs["label"].gpu()
        return (images, labels)
    def iter_setup(self):
        pass

In [33]:
if __name__ == "__main__":
    batch_size = 1
    shapes = ((batch_size, 64, 64, 3),
              (batch_size, 22))
    pipe_test = TFRecordPipelineTest(batch_size=batch_size,
                            num_threads=1,
                            device='gpu',
                            device_id=0)
    with tf.device('/gpu:0'):
        ds_test = dali_tf.DALIDataset(
            pipeline=pipe_test,
            batch_size=batch_size,
            output_shapes=shapes,
            output_dtypes=(tf.float32, tf.int64),
            device_id=0)
        print(ds_test)
        loss, acc1 = model1.evaluate(ds_test, steps=TEST_DATA_SIZE)
        print("Test CNN accuary: ", acc1)

<DALIDataset element_spec=(TensorSpec(shape=(1, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(1, 22), dtype=tf.int64, name=None))>
1282/1282 [==============================] - 5s 3ms/step - loss: 0.7161 - accuracy: 0.8565
Test CNN accuary:  0.8564742803573608


In [34]:
pred_test_cnn = model1.predict(ds_test.take(TEST_DATA_SIZE))

1282/1282 [==============================] - 3s 2ms/step


In [36]:
if __name__ == "__main__":
    batch_size = 1
    shapes = ((batch_size, 64, 64, 3),
              (batch_size, 22))
    pipe_test = TFRecordPipelineTest(batch_size=batch_size,
                            num_threads=1,
                            device='gpu',
                            device_id=0)
    with tf.device('/gpu:0'):
        ds_test = dali_tf.DALIDataset(
            pipeline=pipe_test,
            batch_size=batch_size,
            output_shapes=shapes,
            output_dtypes=(tf.float32, tf.int64),
            device_id=0)
        print(ds_test)
        loss, acc2 = model2.evaluate(ds_test, steps=TEST_DATA_SIZE)
        print("Test ResNet accuary: ", acc2)

<DALIDataset element_spec=(TensorSpec(shape=(1, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(1, 22), dtype=tf.int64, name=None))>
1282/1282 [==============================] - 12s 9ms/step - loss: 0.4175 - accuracy: 0.9470
Test ResNet accuary:  0.9469578862190247


In [37]:
pred_test_resnet = model2.predict(ds_test.take(TEST_DATA_SIZE))

1282/1282 [==============================] - 12s 8ms/step


In [38]:
pred_test = (pred_test_cnn * (acc1/(acc1+acc2)) )+(pred_test_resnet * (acc2/(acc1+acc2)))
true_test = np.concatenate([y for x, y in ds_test.take(TEST_DATA_SIZE)], axis=0)

In [39]:
thresholdAccur(0, pred_test, true_test)
thresholdAccur(0.1, pred_test, true_test)
thresholdAccur(0.2, pred_test, true_test)
thresholdAccur(0.3, pred_test, true_test)
thresholdAccur(0.4, pred_test, true_test)
thresholdAccur(0.5, pred_test, true_test)

thresholde :   -   accuracy_score :  0.9438377535101404  f1_score :  0.9438377535101404
thresholde :  0.1  accuracy_score :  0.7831513260530422  f1_score :  0.8638888888888889
thresholde :  0.2  accuracy_score :  0.8361934477379095  f1_score :  0.8938021022109459
thresholde :  0.3  accuracy_score :  0.87597503900156  f1_score :  0.9129464285714285
thresholde :  0.4  accuracy_score :  0.905616224648986  f1_score :  0.9290076335877863
thresholde :  0.5  accuracy_score :  0.9391575663026521  f1_score :  0.9499013806706114


# multi label

In [40]:
model1 = keras.models.load_model(f'/content/drive/MyDrive/facial_emotion_recognition_using_K-drama_dataset-main/model/cnn_multi.h5')
model2 = keras.models.load_model(f'/content/drive/MyDrive/facial_emotion_recognition_using_K-drama_dataset-main/model/resnet_multi.h5')

In [41]:
class TFRecordPipelineTest(Pipeline):
    def __init__(self, batch_size, num_threads,
                 device = 'cpu', device_id = 0):
        super(TFRecordPipelineTest, self).__init__(batch_size,
                                         num_threads,
                                         device_id)
        self.input = fn.readers.tfrecord(        
             features = {"image_raw": tfrec.FixedLenFeature((), tfrec.string, ""),
                         "label": tfrec.FixedLenFeature([24], tfrec.int64,  24)},
             path = [f_PATH+'tfrecords/multi-label_test.tfrecords'],
             index_path = [f_PATH+'tfrecords/multi-label_test.idx'],
             random_shuffle=False,
             seed = set_seed)
        self.iter = 0
    def define_graph(self):
        inputs = self.input
        images = fn.decoders.image(inputs["image_raw"], device = "mixed", output_type = types.RGB) / 255.
        labels = inputs["label"].gpu()
        return (images, labels)
    def iter_setup(self):
        pass

In [42]:
if __name__ == "__main__":
    batch_size = 1
    shapes = ((batch_size, 64, 64, 3),
              (batch_size, 24))
    pipe_test = TFRecordPipelineTest(batch_size=batch_size,
                            num_threads=4,
                            device='gpu',
                            device_id=0)
    with tf.device('/gpu:0'):
    #with strategy.scope():        
        # Create dataset
        ds_test = dali_tf.DALIDataset(
            pipeline=pipe_test,
            batch_size=batch_size,
            output_shapes=shapes,
            output_dtypes=(tf.float32, tf.int64),
            device_id=0)
        loss, acc1 = model1.evaluate(ds_test, steps=TEST_DATA_SIZE)
        print("Test CNN accuary: ", acc1)

1282/1282 [==============================] - 8s 5ms/step - loss: 0.0738 - accuracy: 0.6147
Test CNN accuary:  0.614664614200592


In [43]:
pred_test_cnn = model1.predict(ds_test.take(TEST_DATA_SIZE))

1282/1282 [==============================] - 3s 3ms/step


In [44]:
if __name__ == "__main__":
    batch_size = 1
    shapes = ((batch_size, 64, 64, 3),
              (batch_size, 24))
    pipe_test = TFRecordPipelineTest(batch_size=batch_size,
                            num_threads=4,
                            device='gpu',
                            device_id=0)
    with tf.device('/gpu:0'):
    #with strategy.scope():        
        # Create dataset
        ds_test = dali_tf.DALIDataset(
            pipeline=pipe_test,
            batch_size=batch_size,
            output_shapes=shapes,
            output_dtypes=(tf.float32, tf.int64),
            device_id=0)
        loss, acc2 = model2.evaluate(ds_test, steps=TEST_DATA_SIZE)
        print("Test ResNet accuary: ", acc2)

1282/1282 [==============================] - 13s 9ms/step - loss: 0.0355 - accuracy: 0.8003
Test ResNet accuary:  0.8003120422363281


In [45]:
pred_test_resnet = model2.predict(ds_test.take(TEST_DATA_SIZE))

1282/1282 [==============================] - 11s 8ms/step


In [46]:
pred_test = (pred_test_cnn * (acc1/(acc1+acc2)) )+(pred_test_resnet * (acc2/(acc1+acc2)))
true_test = np.concatenate([y for x, y in ds_test.take(TEST_DATA_SIZE)], axis=0)

In [47]:
thresholdAccur(0, pred_test, true_test)
thresholdAccur(0.1, pred_test, true_test)
thresholdAccur(0.2, pred_test, true_test)
thresholdAccur(0.3, pred_test, true_test)
thresholdAccur(0.4, pred_test, true_test)
thresholdAccur(0.5, pred_test, true_test)

thresholde :   -   accuracy_score :  0.7418096723868954  f1_score :  0.7418096723868954
thresholde :  0.1  accuracy_score :  0.7581903276131046  f1_score :  0.8871315600287561
thresholde :  0.2  accuracy_score :  0.8260530421216848  f1_score :  0.9127382146439317
thresholde :  0.3  accuracy_score :  0.8517940717628705  f1_score :  0.9239494715132767
thresholde :  0.4  accuracy_score :  0.8712948517940717  f1_score :  0.9357992073976222
thresholde :  0.5  accuracy_score :  0.859594383775351  f1_score :  0.9308655416439847
